# Understanding Academic Data

This tutorila will guide you through the analysis of Academic Data with the purpose of having a better undestanding of what students are doing inside the program (eg. which courses they find difficult, which courses are taken together, etc.).

As a part of this tutorial, you will also learn useful skills such as wrangling data to obtain a processable dataset, analyze groupings through clustering and to analyze sequential data through process mining. Let's start.

## Step 0: Setting up the notebook

This step is only needed if you are running the code in this notebook.  It is not necesary if you are running the code in your own R-Studio script.  Its purpose is to set the size of the graphs and charts.

In [1]:
options(repr.plot.width = 6, repr.plot.height = 6)

## Step 1: Importing the data

The data that we will use throughout this tutorial is the academic history of a higher education institution. 

In the last tutorial we learned how to import data from a CSV file.  In this tutorial we will import data from an Microsoft Excel file.  Given the extensive use of Excel, you are bound to find datasets that are stored in this format. 

Run the following code.

NOTE: The dataset is very large.  It could take between 2 to 5 minutes to read all the data. Be patient.

In [2]:
library(readxl)
academic_data <- read_excel("AnonymizedAcademicData.xlsx")

<font color="darkblue">**Explanation:**
<ul>
<li>The first line is loading the *readxl* library that have the command to import Excel files.
<li>The second line use the *read_excel* function from the file (the file should be in the same directory as the code) and store it in the variable academic_data.
</ul>
</font>

Now, let's take a look at the structure of the dataset

In [3]:
head(academic_data)

record,student_code,course_code,grade,year,semester,section,status
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,b2073618,1429e510,6.73,1978,1S,001,AP
2,b2073618,f256f99e,0.00,1986,2S,001,RP
3,b2073618,d7d5c424,5.30,1984,2S,001,RP
4,b2073618,27fe8648,3.75,1984,1S,001,RP
5,b2073618,94906395,8.07,1982,1S,001,AP
6,b2073618,f37c3266,5.70,1986,1S,001,RP


As you can see, the structure is very simple:
* record: Sequential number of the record
* student_code:  Anonymized student identification
* course_code: Anonymized course identification
* grade: Final grade obtained by the student in the course.  The grades go between 0 and 10 with decimal values.
* year: Year in which this course was taken
* semester: The semester in which the course was taken (1S, 2S or 3S)
* section: Section identifier for the course
* status: If the course was approved (AP) and failed (any other code).

## Step 1: Course difficulty

The grades obtained by the students give us information about how difficult they find the courses much better than any survey that we can conduct.  By obtaining a grade stringency metric, such as Beta (see this week reading), we can get a good picture on how more difficult (or easier) student find this course compared with the rest of the courses that they have taken.  

To conduct this analysis, we need first to manipulate the dataset to obtain the GPA of each student and the BETA values for each course.  Let start by calculating each student GPA or mean grade.

In [4]:
library(dplyr)
grouped_data <- group_by(academic_data,student_code)
student_gpa <- summarize(grouped_data,gpa=mean(grade))
head(student_gpa)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



student_code,gpa
<chr>,<dbl>
0016a4dc,7.266094
008744d6,7.251719
009aaf5b,7.263000
009d1332,6.688281
00cdff76,4.488235
01143eb5,7.221389


<font color="darkblue">**Explanation:**
<ul>
<li>The first line is loading the *dplyr* library that help us to easily manipulate the dataset.  More information about this library can be found <a href="https://genomicsclass.github.io/book/pages/dplyr_tutorial.html">here</a>.
<li>In the second line we use the *group_by* function.  We are telling R that it should take the academic_data dataframe and group it by the student_code.  This grouping will be sotred in the grouped_data variable.  There is no visible change to the dataset at this stage.
<li>The third line use the *summarize* function.  In this function we are telling R that we want to use the grouped_data and calculate the mean of the grade for each group (in this case, each group is a given student) and store it in the variable *gpa* of a new dataframe.  We store this dataframe in the variable *student_gpa*.
<li>The final line just show the first lines of this new dataframe.  As we wanted, now we have the mean grade (GPA) for each student.
</ul>
</font>

It is common that the we use several functions of the dplyr libary one after the other in sequence.  In our previous example, we are using first *group_by* and then we are using *summarize*.  To simplify this concatenation of functions, R provides the %>% operator that just send the current result as input of the next function.  For example the same code as before can be expressed as:

In [5]:
student_gpa <- 
  academic_data %>%
  group_by(student_code) %>%
  summarise(
    gpa = mean(grade)
  )
head(student_gpa)

student_code,gpa
<chr>,<dbl>
0016a4dc,7.266094
008744d6,7.251719
009aaf5b,7.263000
009d1332,6.688281
00cdff76,4.488235
01143eb5,7.221389


<font color="darkblue">**Explanation:**
<ul>
<li>The first line contains the name of the variable in which we will store the result of the transformation.
<li>In the second line we are specifying the *academic_data* as the dataframe that we will use and we send to the next function.
<li>The third line groups the dataframe by each *student_code* and we send this result to the next function.
<li>The fourth, fifth and sixthl lines received the grouped dataframe and call the summarize function.  The parameter for the function is given in the fifth line when we say that a *gpa* column will store the mean grade for each student. 
<li>Finally we show the result of the operation.
</ul>
</font>

Next, we create function to return the gpa of any student given its student_code.  We will use this function later

In [6]:
get_gpa<-function(code)
{
  gpa<- student_gpa %>% filter(student_code %in% c(code))
  gpa$gpa
}

<font color="darkblue">**Explanation:**
<ul>
<li>In the first line we let R know that we want to create a function that is called *get_gpa*.  This function will receive a parameter that is called *code*.
<li>The thrid line use the filter function of dplyr.  This function select from the *student_gpa* dataframe only that row in which the *student_code* is equal to the code passed as a paremeter to the function.
<li>The fourth line returns the gpa column of the extracted row.  This is the GPA of the student.
</ul>
</font>

Let's test the function

In [7]:
get_gpa('0016a4dc')

[1] 7.266094

<font color="darkblue">**Explanation:**
<ul>
<li>To call the function we only need to write its name and to pass the needed parameters.  We obtain back the GPA of the student *00164dc*
</ul>
</font>

Now, we apply this function to calculate the deviation from the grade of each student in each course compared with their GPA.  

NOTE: This code will take a while to run (between 2 and 5 minutes).  Be patient

In [8]:
academic_data$student_gpa<-sapply(academic_data$student_code,get_gpa)
academic_data <- academic_data %>% mutate(adjusted_grade=grade-student_gpa)
head(academic_data)

record,student_code,course_code,grade,year,semester,section,status,student_gpa,adjusted_grade
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,b2073618,1429e510,6.73,1978,1S,001,AP,5.21375,1.51625
2,b2073618,f256f99e,0.00,1986,2S,001,RP,5.21375,-5.21375
3,b2073618,d7d5c424,5.30,1984,2S,001,RP,5.21375,0.08625
4,b2073618,27fe8648,3.75,1984,1S,001,RP,5.21375,-1.46375
5,b2073618,94906395,8.07,1982,1S,001,AP,5.21375,2.85625
6,b2073618,f37c3266,5.70,1986,1S,001,RP,5.21375,0.48625


<font color="darkblue">**Explanation:**
<ul>
<li>The first line uses the function *sapply* to apply the *get_gpa* function to the *student_code* value of each line in the academic_data dataframe.  We store the result of this calculation in a new column named *student_gpa* 
<li>The second line uses the *mutate* function of the dplyr package to create a new column named *adjusted_grade* that is the difference between the grade that the student obtained in the course minus their GPA (*student_gpa*)
<li>With the third line you can see the newly changed dataset with the *adjusted_grade* column that we will use to calculate the BETA of the course.
</ul>
</font>

With the Adjusted Grade obtained, now it is easy to calculate the Beta of each course.  We just need to obtain the mean adjusted grade for all the students that have taken a given course.  The following code will do the trick:

In [9]:
course_beta <- 
  academic_data %>%
  group_by(course_code) %>%
  summarise(
    beta = mean(adjusted_grade)
  ) 
head(course_beta %>% arrange(desc(beta)))

course_code,beta
<chr>,<dbl>
2cd89574,4.718298
5e846c67,4.207429
318a144a,3.986316
0a486675,3.936316
b02ada88,3.382913
3ff8c62c,3.379167


<font color="darkblue">**Explanation:**
<ul>
<li>We apply first the *group_by* function, to aggregate the dataset by *course_code*.  In this way each operation that we will perform later will be done for each individual course. 
<li>Then we use the *summarise* function to calculate the *beta* of the course as the mean of the *adjusted_grade* of the students.
<li>We store this result in the *course_beta* dataframe and we show the first rows in descending order using the *arrange* function of the dplyr package.
</ul>
</font>

The beta was calculated for all courses, even if they just got one student.  We want to only consider courses that have at least 30 students.

In [10]:
large_courses <- 
  academic_data %>% 
  count(course_code) %>% 
  filter(n>30)
head(large_courses)

course_code,n
<chr>,<int>
010d3db1,64
01695174,67
01bd7b1c,1219
03071022,37
031b1fda,85
031d5c74,501


<font color="darkblue">**Explanation:**
<ul>
<li>Here we use the dplyr functions to first *count* the number of records that have a given *course_code* and then use *filter* only those that have more than 30 records (meaning students).
<li>Then we store this in the *large_courses* dataframe and show the first elements.
</ul>
</font>

To get an idea of the distribution of Beta values among courses, we create a boxplot.  We only use those courses with more than 30 students.

In [11]:
betas<-course_beta %>% filter(course_code %in% large_courses$course_code)
boxplot(betas$beta,main="Distribution of Course Difficulty (beta)",ylab="Difficulty - Larger -> Easier" )

<font color="darkblue">**Explanation:**
<ul>
<li>First, we select from the *course_beta* dataframe only those courses that are in *large_courses* and store those in the *betas* dataframe.
<li>Then we create a boxplot with the *beta* column of the *betas* dataframe.
<li>As we can see that the average difficulty is around 0.  Most courses (50%) are between -0.5 and 0.5 beta.  However, there are several courses easier or harder than the average.
</ul>
</font>

To better explore the course difficulty we create a function to obtain the distribution of adjusted grades for a given course.

In [12]:
get_beta_distribution<- function(code)
{
  betas<- academic_data %>% filter(course_code %in% c(code))
  betas$adjusted_grade
}

<font color="darkblue">**Explanation:**
<ul>
<li>This function called *get_beta_distribution* has only one parameter that is called *code* that will receive the code of the course.
<li>In this function we obtain, through the filter function, all the rows in the academic data dataframe in which the course code is equal to the provided code will be returned and stored in the *betas* dataframe.  Then we just return the adjusted_grade of all those records.
</ul>
</font>

First, we would want to explore difficult courses.  For that we list those courses.

In [13]:
difficult_large_courses <- 
  course_beta %>%
  filter(course_code %in% large_courses$course_code) %>%
  filter(beta < - 0.5)
head(difficult_large_courses %>% arrange(beta),10)


course_code,beta
<chr>,<dbl>
533a6598,-2.230883
f78bb17d,-2.046877
6a5911d3,-1.915115
d32d5edd,-1.810224
a42f35f5,-1.773848
9d4c41be,-1.741703
748f5b8e,-1.467070
12a520d3,-1.287482
a33ff6dc,-1.240471


<font color="darkblue">**Explanation:**
<ul>
<li>We use dplyr functions to first filter the course_beta that are in the large_courses dataframe.  Then we filter only those that have a beta lower than -0.5.
<li>We present this list the top 10 in descending order.
</ul>
</font>

Out of the large courses we will explore '533a6598' (The aparent most difficult course).

In [14]:
course<-'533a6598'
hist(get_beta_distribution(course),main=paste("Beta Distribution for course",course),xlab="Adjusted Grade",breaks=20)
abline(v=mean(get_beta_distribution(course)),col="red")

<font color="darkblue">**Explanation:**
<ul>
<li>We store the code of the course in the *course* variable.
<li>The second line present the histogram of the adjusted grade values for that course.
<li>The third line of code creates a red vertical line at the beta value of the course (the mean adjusted grade). 
</ul> 
</font>

It seems that in this course, most students receive a much lower grade than their usual GPA.  In average, less than 2 points over 10.  

Let's explore another course in the top list.

In [15]:
course<-'27fe8648'
hist(get_beta_distribution(course),main=paste("Beta Distribution for course",course),xlab="Adjusted Grade",breaks=20)
abline(v=mean(get_beta_distribution(course)),col="red")

Here we see a cleared distribution with a skewed tail towards negative values, the signature of a difficult course.

Now we will explore the apparently easy courses:

In [16]:
easy_large_courses <- 
  course_beta %>%
  filter(course_code %in% large_courses$course_code) %>%
  filter(beta > 0.5)
head(easy_large_courses %>% arrange(desc(beta)),10)

course_code,beta
<chr>,<dbl>
f52145da,2.641334
a113d4af,2.372671
ac30450e,2.108558
ba02f7ce,2.062132
30027735,2.050043
0a53d75c,1.815479
b6d36ddc,1.717625
103c4631,1.685786
87824b85,1.633114


<font color="darkblue">**Explanation:**
<ul>
<li>Very similar to the difficult_large_courses calculation, this time we select large courses that have a beta larger than 0.5
</ul> 
</font>

We will explore two courses from this list

In [17]:
course<-'f52145da'
hist(get_beta_distribution(course),main=paste("Beta Distribution for course",course),xlab="Adjusted Grade",breaks=20)
abline(v=mean(get_beta_distribution(course)),col="red")

In [18]:
course<-'d57b181b'
hist(get_beta_distribution(course),main=paste("Beta Distribution for course",course),xlab="Adjusted Grade", breaks=20)
abline(v=mean(get_beta_distribution(course)),col="red")

This visualizations could help a teachers reflect on the grade practices in their courses or could help students to better assess the difficulty of the courses that they select.

## Step 2: Clustering

The next step will guide us through the clustering techniques.  These techniques search for similarities in the data to group similar entities.  These technique could help humans to make sense of multivariate data in an easier way.  To learn more about clustering techniques, visit this <a href="https://home.deib.polimi.it/matteucc/Clustering/tutorial_html/">site</a>.

Also through these step we will learn how to change the shape of our data and how to deal with missing values.

First, to avoid long computation times we will select just 8 courses that are known to be common among students at this university.

In [19]:
common_courses<-c("c66fa9f7","a65b6acf","87824b85","1b40bae0","47f79e31","ee143666","0ad99f1e","44e3b71e",
                  "5647927e","2c39cee7","3f0c9401","45480b29","44ecaefc","5fcb1de4","dd6dd97d","3ef98742")
selected_data <-
  academic_data %>%
  filter(course_code %in% common_courses)

<font color="darkblue">**Explanation:**
<ul>
<li>First we create a list with the code of all the common courses.
<li>Then se use filter to select just those records in which its course_code is among the common courses codes.
</ul> 
</font>

There still around 35.000 records after this selection

In [20]:
length(selected_data$student_code)

[1] 35090

We now want to group the students according to their performance in those courses.  To do it we need to manipulate the shape of the data in order to feed it to the cluster algorithms.

Currently the data is what is known as long-format.  Each row in the dataframe represent one student taking one course.  

In [21]:
head(selected_data)

record,student_code,course_code,grade,year,semester,section,status,student_gpa,adjusted_grade
<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
26,b2073618,0ad99f1e,6.11,1983,2S,001,AP,5.213750,0.8962500
32,b2073618,0ad99f1e,4.71,1983,1S,001,RP,5.213750,-0.5037500
60,b2073618,c66fa9f7,6.09,1978,1S,001,AP,5.213750,0.8762500
86,535cc5a9,c66fa9f7,6.28,1979,2S,001,AP,3.486618,2.7933824
93,535cc5a9,47f79e31,0.00,1987,1S,001,PF,3.486618,-3.4866176
97,535cc5a9,47f79e31,3.19,1988,2S,001,RP,3.486618,-0.2966176


To be able to cluster, we need to create a dataset where each row represent an student and each column represent the average grade of the student in each one of the courses (some students repeat a course and have several grades for the same course).  This is known as wide-format. 

To make this changes, we use a library called reshape.  This provide functions such as *cast* to change from long to wide formats and *melt* to change from wide to long.

NOTE: This code take a while to execute (2 to 5 minutes).  Again, be patient.

In [22]:
library(reshape)
casted_data <- cast(selected_data,student_code~course_code,value='grade',mean)
length(casted_data$student_code)
head(casted_data)


Attaching package: ‘reshape’

The following object is masked from ‘package:dplyr’:

    rename



[1] 2478

student_code,0ad99f1e,1b40bae0,2c39cee7,3ef98742,3f0c9401,44e3b71e,44ecaefc,45480b29,47f79e31,5647927e,5fcb1de4,87824b85,a65b6acf,c66fa9f7,dd6dd97d,ee143666
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0016a4dc,7.650,6.06,7.66,9.25,NaN,8.00,6.95,6.8600,6.880000,7.21,NaN,NaN,7.97,6.330,NaN,NaN
008744d6,8.150,6.60,6.40,8.25,NaN,7.55,9.00,7.3000,6.860000,7.43,NaN,NaN,6.36,6.130,NaN,NaN
009aaf5b,6.000,7.90,6.40,6.80,NaN,6.10,7.85,6.3500,7.500000,6.40,NaN,7.7,6.25,6.050,NaN,8.600
009d1332,4.275,6.05,6.00,7.95,3.783333,3.85,7.40,7.7500,5.666667,6.25,7.60,7.9,6.50,6.825,6.675,7.150
00cdff76,NaN,1.80,5.00,NaN,6.075000,7.70,NaN,1.9625,NaN,6.70,8.65,6.2,6.05,5.900,7.450,6.625
01143eb5,6.030,8.60,7.66,NaN,NaN,NaN,6.86,6.6000,6.310000,7.76,NaN,NaN,7.70,7.830,NaN,NaN


<font color="darkblue">**Explanation:**
<ul>
<li>First we load the reshape library.
<li>Then we use the *cast* function to convert from long to wide.  We need to specify how this new dataframe will be structured by the formula student_code~course_code, meaning that we will use students as rows and courses as columns.  The value parameter specify which column we will use to fill the new dataframe.  We will use the *grade*.  Because there could be several rows that have the same student and course, we need to specify which function we will use to aggreate the different values.  In this case we specify that is the mean function (in this case the mean of the grades that the student got into the same course).
<li>Finally we show how many students remain in this dataset (2478) and the structure of this new dataset (stored in casted_data).  It is as we desired, each row is a different student and each column is a different course.  The values are the average grade of the student in that course.
</ul> 
</font>

Now we have the data in the shape that we need, however, we can see that there is a lot of cells in which the value is "NaN", meaning that there that specific student never took that specific course.  This is a natural consequense of the nature of the data and the casting operation that we did.

The existence of this missing data is a common problem for a lot of statistical and machine learning algorithms. We need to find a solution.

There are several ways in which we can deal with missing values.  You can read <a href="https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4">here</a> a very good introductory guide.  In this tutorial we will focus on removal.

First we will explore removal or deletion.  That means to eliminate all the records that have missing values in any of its columns.  In the case of this dataset, it will be equivalent to remove all the students that have not taken all the 16 courses listed.


In [23]:
cleaned_data <- na.omit(casted_data) 

<font color="darkblue">**Explanation:**
<ul>
<li>This function *na.omit* delete all the rows that have at least one missing value.  
</ul> 
</font>

In [24]:
length(cleaned_data$student_code)
head(cleaned_data)

[1] 526

,student_code,0ad99f1e,1b40bae0,2c39cee7,3ef98742,3f0c9401,44e3b71e,44ecaefc,45480b29,47f79e31,5647927e,5fcb1de4,87824b85,a65b6acf,c66fa9f7,dd6dd97d,ee143666
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,009d1332,4.275,6.050,6.000,7.95,3.783333,3.85,7.40,7.750,5.666667,6.25,7.6,7.9,6.500,6.825,6.675,7.15
8,01368778,6.100,4.900,6.350,6.30,5.516667,7.50,4.90,5.300,6.400000,6.00,7.5,8.5,6.000,6.800,6.200,6.50
10,015ba60f,7.100,6.525,6.050,9.25,6.700000,7.60,6.45,5.975,6.550000,4.15,10.0,8.1,6.575,7.600,7.950,7.20
22,028bebee,7.650,7.850,5.775,7.40,6.150000,7.00,7.50,8.150,6.100000,6.95,7.8,8.7,6.350,7.000,6.000,8.25
28,03303d2c,6.150,7.150,6.250,8.45,9.150000,6.75,6.20,6.550,7.025000,8.00,9.1,9.3,7.150,5.675,7.950,8.00
43,04d664ca,6.550,6.800,6.500,8.25,6.100000,8.00,8.50,6.550,6.450000,6.65,7.5,9.4,6.000,6.850,7.950,9.50


After removing the students that have not taken all the courses, we end up with 526 students.  A number big enough to perform the clustering analysis.

Then we would want to group the students according to their performance in the different courses.  For that we need to use a clustering algorithm.
For demostration purposes we will use several in sequence.

First we will start with the easiest to understand, <a href="https://blog.easysol.net/machine-learning-algorithms-3/">K-Means</a>.

In [25]:
library(cluster)
grade_vector<-subset(cleaned_data, select = -c(student_code))
fit<-pam(grade_vector,3)
aggregate(grade_vector,by=list(fit$cluster),FUN=mean)

Group.1,0ad99f1e,1b40bae0,2c39cee7,3ef98742,3f0c9401,44e3b71e,44ecaefc,45480b29,47f79e31,5647927e,5fcb1de4,87824b85,a65b6acf,c66fa9f7,dd6dd97d,ee143666
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,6.557555,6.040343,6.542913,7.500000,4.679790,6.190576,6.729673,6.301246,6.246340,6.086371,6.766589,8.065654,6.262461,6.656604,6.737617,7.178505
2,6.225888,5.420537,6.004061,7.350508,6.383947,6.316794,6.395389,5.540186,5.539763,6.044036,7.118909,8.119416,5.862521,6.387589,6.725465,7.186633
3,7.328941,6.728529,6.940916,7.887838,7.388476,7.605968,7.424437,6.826014,6.862688,7.084122,8.147185,8.792005,6.740728,7.369662,7.754730,7.754767


<font color="darkblue">**Explanation:**
<ul>
<li>First we load the library *cluster* to load all the relevant functions.
<li>Second we remove the student_code from the dataset with the command *subset*.  We store the result in the variable grade_vector.
<li>Third we apply a robust version of k-means through the command *pam*.  We need to specify the number of groups that we want to create.  In this case we specify 3.
<li>Then we use the function aggregate to show the centroids of the different clusters, in this case is the average grade that each student in the cluster has in the different courses.  For example we expect that a student in the cluster 1 has a lower score in '3f0c9401' than a student in the cluster 3.
</ul> 
</font>

We know have classified the student in different clusters

In [26]:
clustered_students <- data.frame(student=cleaned_data$student_code, cluster=fit$cluster)
head(clustered_students)

,student,cluster
,<fct>,<int>
4,009d1332,1
8,01368778,2
10,015ba60f,3
22,028bebee,3
28,03303d2c,3
43,04d664ca,3


We can see how many students belong to each cluster.

In [27]:
table(clustered_students$cluster)


  1   2   3 
107 197 222 

It is difficult to have a visual representation of the clusters because they belong to a 8-dimensional space.  The best thing we can do is to create a two dimmensional representation using Principal Component Analysis to get the best discrimination in two dimensions.

In [28]:
clusplot(grade_vector, fit$cluster, color=TRUE, shade=FALSE, labels=4, lines=0)

<font color="darkblue">**Explanation:**
<ul>
<li>The cusplot function plot the values in the dataframe against two dimensions obtained from Principal Component Analysis.  The clusters are represented by the elipses. 
<li>It can be seen from the graph that Cluster 1 comprise the middle points, Cluster 2 the left points and Cluster 3 the right points.
</ul> 
</font>

To explore the other clustering techniques, we will reverse our analysis.  We will cluster the courses based on the grades that students get in them. To do this, we need to transpose our dataframe.

In [29]:
library(data.table)
n <- names(cleaned_data[,-1])
course_data<- transpose(cleaned_data[,-1])
rownames(course_data)<-n


Attaching package: ‘data.table’

The following object is masked from ‘package:reshape’:

    melt

The following objects are masked from ‘package:dplyr’:

    between, first, last



<font color="darkblue">**Explanation:**
<ul>
<li>First we load the data.table library that contains the transpose function.
<li>Second we store the name of the courses in the variable n.  We discard the first element, because that is the student_code column name.
<li>We use the *transpose* function to flip the dataframe.  We do not use the first column (it is the student_code)
<li>Finally, we assign the name of the courses as the new name of the rows.
</ul> 
</font>

In [30]:
head(course_data)
length(course_data$V1)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V517,V518,V519,V520,V521,V522,V523,V524,V525,V526
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0ad99f1e,4.275000,6.100000,7.100,7.650,6.15,6.55,6.30,8.85,2.900000,6.550000,...,6.05,5.575,7.20,8.55,7.55,7.10,7.350,8.150000,7.450,7.550
1b40bae0,6.050000,4.900000,6.525,7.850,7.15,6.80,6.00,8.70,2.983333,7.200000,...,6.40,6.850,6.95,8.75,7.70,0.00,6.250,6.100000,6.175,6.325
2c39cee7,6.000000,6.350000,6.050,5.775,6.25,6.50,6.60,8.75,4.062500,6.000000,...,6.00,6.000,6.25,6.05,6.85,6.00,7.175,7.550000,6.000,6.000
3ef98742,7.950000,6.300000,9.250,7.400,8.45,8.25,7.95,9.15,7.900000,8.550000,...,8.00,7.000,7.15,8.90,8.40,7.75,6.700,7.450000,6.900,7.950
3f0c9401,3.783333,5.516667,6.700,6.150,9.15,6.10,6.25,8.20,3.475000,4.466667,...,6.75,6.200,7.90,7.80,7.35,6.50,6.950,5.683333,9.550,8.400
44e3b71e,3.850000,7.500000,7.600,7.000,6.75,8.00,7.55,9.25,6.500000,5.425000,...,6.05,6.750,7.40,6.50,7.55,7.35,9.400,6.200000,9.800,6.000


[1] 16

Here we can see the transposed dataset.  It has 16 rows corresponding to the 16 courses.  It has 526 columns corresponding to the 526 students.

Now we will use Hierarchical clustering to discover relationships between the courses.

In [31]:
d <- dist(course_data, method = "euclidean") 
fit <- hclust(d, method="ward.D")
plot(fit, main="Cluster Diagram",xlab="Courses")
groups <- cutree(fit, k=4) 
rect.hclust(fit, k=4, border="red")

<font color="darkblue">**Explanation:**
<ul>
<li>The first line calculate the distance between the vectors for each course using euclidean distance.
<li>The second line create the relationships between the courses according to the distance.  Several methods can be used to create the clusters. We use the ward method.
<li>The third line plot the dendogram, that is the relationship between the different courses.  
<li>The fourth line create 4 clusters by cutting the relationships at a certain height.
<li>Finally, the fifth line represent this clusters graphically.
</ul> 
</font>

Now we will use another clustering technique, called Fuzzy C-Clustering in which one element could have different levels of attachment to each cluster.

We need to download and install the *ppclust* libarary.

NOTE: This could take 10 minutes. Be patient.

In [51]:
install.packages("ppclust")
library(ppclust)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependencies ‘lhs’, ‘inaparc’



In [52]:
res.fcm <- fcm(course_data, centers=4)
as.data.frame(res.fcm$u)

,Cluster 1,Cluster 2,Cluster 3,Cluster 4
,<dbl>,<dbl>,<dbl>,<dbl>
0ad99f1e,0.2903737,0.12887895,0.2903737,0.2903737
1b40bae0,0.3002075,0.09937736,0.3002075,0.3002075
2c39cee7,0.3000400,0.09987985,0.3000400,0.3000400
3ef98742,0.1408385,0.57748448,0.1408385,0.1408385
3f0c9401,0.2858079,0.14257622,0.2858079,0.2858079
44e3b71e,0.2762518,0.17124446,0.2762518,0.2762518
44ecaefc,0.2657722,0.20268328,0.2657722,0.2657722
45480b29,0.3009430,0.09717088,0.3009430,0.3009430
47f79e31,0.2995064,0.10148073,0.2995064,0.2995064


<font color="darkblue">**Explanation:**
<ul>
<li>First we use the function fcm to create 4 fuzzy clusters for the courses.
<li>Once the clustering is completed, we show the degree of belonging of each course to a cluster.  For example, course 87824b85 has a strong degree of belonging to Cluster 4 and very low belonging to the other clusters.  On the other hand, course 0.2903737 belongs equally to Cluster 1, 2 and 3.  This is a better way to understand the relationship between courses.
</ul> 
</font>

## Step 3: Sequence Analysis

Sequence analysis use the temporal aspect of data to provide insight on how events occurs over time. For this dataset we are interested in three things:
* How many courses each student take during each one of the semesters
* How many students take courses during each semester
* What is the sequence of number of courses that the student take

Before we can start answering these questions, we need to convert the temporal information in the dataset, expressed as year and semester, as something we could use as the the number of semester for each student.

Let's start by first descarting all the courses that were not taken during the 1st or 2nd semester (noise in the data).  Also, to separate each individual semester, we create a variable that contain the concatenation of the student_code, the year and semester.  This identifier is unique for all the courses taken during the same semester.  We also only take into account records after 2005 to reduce computational time.

In [33]:
academic_sequence<-academic_data %>% 
  mutate(sequence_id=paste(student_code,year,semester,sep="_")) %>%
  filter(semester %in% c("1S","2S")) %>% 
  filter(year > 2000)

<font color="darkblue">**Explanation:**
<ul>
<li>We use mutate to create the new column called sequence_id with the concatenation (*paste*) of *student_code*,*year* and *semester*.  We use the character "_" to separate the values.
<li>Then we use the filter function to only collect those records where the semester is either "1S" or "2S"
</ul> 
</font>

Now we create a new dataset that will list all the courses that were taken into the same semester in one of its columns.  To do it we use functions from the dplyr package.

In [34]:
semesters <- academic_sequence %>%
  group_by(student_code,year,semester) %>%
  summarize(
    sequence_id=max(sequence_id),
    size= n(),
    courses = paste(as.character(course_code), collapse = ';')
  )

<font color="darkblue">**Explanation:**
<ul>
<li>First we group the information by student_code, year and semester.  This information is unique for all the courses taken during the same semester.
<li>Then we create three columns through the summarize function: sequence that is the unique identifier of the semester for the student, the size that is the number of courses in that semester and courses that is a list of the identifier of the courses divided by ";"
</ul> 
</font>

Now we create a function to obtain the number of semester given the unique combination of student_code, year and semester present in *sequence_id*.  This is a complicated function, do not worry if it takes time to understand.

In [35]:
get_semester_order<-function(sequence_id)
{
  components<-strsplit(sequence_id, "_")
  id<-components[[1]][1]
  year<-components[[1]][2]
  semester<-components[[1]][3]
  records<-academic_sequence[academic_sequence$student_code %in% c(id),]
  firstyear<-min(records$year)
  sem<-min(records[records$year==firstyear,]$semester)
  nofSemesters=strtoi(year)-strtoi(firstyear)
  nofSemesters=(nofSemesters*2)+1
  if(sem %in% c("1S") & semester %in% c("2S")){
    nofSemesters=nofSemesters+1
  }
  if(sem %in% c("2S") & semester %in% c("1S")){
    nofSemesters=nofSemesters-1
  }
  nofSemesters
}

<font color="darkblue">**Explanation:**
<ul>
<li>Fist we split the sequence_id into its components: id, year and semester
<li>Then we obtain all the records belonging to the student_code and store them in the variable recrods
<li>We then obtain the first year for that students by taken the minimum value of the column year
<li>Then we obtain the semester in first semester of the first year
<li>Now we calculate how many semester has passed since the first year and the year been consulted
<li>If the student started in the first semester and the semester been consulted is the second semester, add 1 to the number of semesters
<li>If the student started in the second semester and the semester been consulted is the first semester, decrese 1 to the number of semesters
<li>Then we return the number of semesters (*nofSemesters*)
</ul> 
</font>

Now we apply this function for each semester in the dataframe.

NOTE: This calculation will take a while (5 minutes).  Be patient.

In [36]:
semesters$semester_id<-sapply(semesters$sequence_id,get_semester_order)

<font color="darkblue">**Explanation:**
<ul>
<li>This apply the function get_semester_order to each one of the rows of the *semesters* dataframe and store the result in the *semester_id* column.
</ul> 
</font>

With this new dataset we can answer question one: How many courses the students take per semester

In [37]:
courses_taken <-
            semesters %>%
            group_by(semester_id) %>%
            summarize(mean_courses=mean(size))
plot(courses_taken)

<font color="darkblue">**Explanation:**
<ul>
<li>We first group the dataframe by the number of semester (semester_id) and then we obtain the mean of the number of courses (size).
<li>From the result we can see that students usually take 4.5 courses during the first semesters and then 4 around the 10 semester, and then it goes down to 3.
</ul> 
</font>

In [38]:
students_semester <-
            semesters %>%
            group_by(semester_id) %>%
            summarize(students=n())
plot(students_semester)

<font color="darkblue">**Explanation:**
<ul>
<li>Here we can see how the number of students taken a given semester decrease with the number of semesters.
</ul> 
</font>

Now we will apply sequence mining to see if there are meaningful patterns of courses been taken together.  For this we will use the *TraMineR* library.

NOTE: Installation could take a while (3 minutes)

In [39]:
install.packages("TraMineR")
library(TraMineR)

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)

TraMineR stable version 2.0-12 (Built: 2019-09-14)
Website: http://traminer.unige.ch
Please type 'citation("TraMineR")' for citation information.



We need to change the structure of the data in the semesters dataframe for it to be read from the TraMineR package.  The sequence of events, in this case the number of courses taken each semester, should be a list of events separated by a dash.  Example: 4-5-8-4-6.

In [40]:
sequence_data<-semesters %>% 
  group_by(student_code) %>%
  summarize(
    sequence = paste(as.character(size), collapse = '-')
  )
head(sequence_data)

student_code,sequence
<chr>,<chr>
009d1332,5-4-4-4-4-3-2-2-3-3-4-5-5-5-4-3
00cdff76,6-6-4-6-2
01211139,5-5-3-1-5-5-6-6-5-4-3-3-2-2-1
01368778,5-5-5-4-6-4-5-5-5-6-3
01509981,4-5-6-4-4-6-7-6-6-4-2
015ba60f,5-4-5-5-3


<font color="darkblue">**Explanation:**
<ul>
<li>We group the dataframe by each student
<li>Then we take the number of courses (size) and we join them into a long string separted by dash.
<li>We store this new dataset in the variable sequence_data.  As you can see, if follows the required format.
</ul> 
</font>

Now we read the sequence.

In [41]:
sqd<-seqdef(sequence_data$sequence)

 [>] found missing values ('NA') in sequence data
 [>] preparing 1830 sequences
 [>] coding void elements with '%' and missing values with '*'
 [>] 10 distinct states appear in the data: 
     1 = 1
     2 = 2
     3 = 3
     4 = 4
     5 = 5
     6 = 6
     7 = 7
     8 = 8
     9 = 9
     10 = 10
 [>] state coding:
       [alphabet]  [label]  [long label] 
     1  1           1        1
     2  2           2        2
     3  3           3        3
     4  4           4        4
     5  5           5        5
     6  6           6        6
     7  7           7        7
     8  8           8        8
     9  9           9        9
     10  10          10       10
 [>] 1830 sequences in the data set
 [>] min/max sequence length: 1/23


For reference in the next visualizations, we generate the legend for the graphs

In [42]:
seqlegend(sqd, fontsize = 1.3)

 [!] In IRkernel::main() : fontsize is deprecated, use cex instead.


Now we visualize the first 10 sequences

In [43]:
seqiplot(sqd, withlegend = F, title = "Index plot (10 first sequences)", border = NA)

 [!] In IRkernel::main() : title is deprecated, use main instead.
 [!] In IRkernel::main() : withlegend is deprecated, use with.legend instead.


Now we visualize the 10 most common sequences, with bar width proportional to the frequencies 

In [44]:
seqfplot(sqd, withlegend = T, border = NA, title = "Sequence frequency plot")

 [!] In IRkernel::main() : title is deprecated, use main instead.
 [!] In IRkernel::main() : withlegend is deprecated, use with.legend instead.


Now we visualize the distribution of values for different semesters

In [45]:
seqdplot(sqd, withlegend = T, border = NA, title = "State distribution plot")

 [!] In IRkernel::main() : title is deprecated, use main instead.
 [!] In IRkernel::main() : withlegend is deprecated, use with.legend instead.


Finally, we will cluster the students according to their sequence.  To do it we use the agnes hierarquical cluster algorithm.

In [49]:
submat <- seqsubm(sqd, method = "TRATE")
dist.om1 <- seqdist(sqd, method = "OM", indel = 1, sm = submat)
clusterward1 <- agnes(dist.om1, diss = TRUE, method = "ward")
cl1.4 <- cutree(clusterward1, k = 4)
cl1.4fac <- factor(cl1.4, labels = paste("Type", 1:4))
seqdplot(sqd, group = cl1.4fac, border = NA)

 [>] creating substitution-cost matrix using transition rates ...
 [>] computing transition probabilities for states 1/2/3/4/5/6/7/8/9/10 ...
 [>] 1830 sequences with 10 distinct states
 [>] checking 'sm' (one value for each state, triangle inequality)
 [>] 1579 distinct sequences
 [>] min/max sequence length: 1/23
 [>] computing distances using the OM metric
 [>] elapsed time: 3.87 secs


<font color="darkblue">**Explanation:**
<ul>
<li>The first two lines ares used to calculate the distance between the vectors of different students.
<li>The third lines applies the Agnes algorithm to find the relationships between students.
<li>The fourth line divide the hierquical cluster into 4 groups
<li>The fifth line convert the cluster name into a factor
<li>The sixth line create the distribution of values for different semester, but one graph for each cluster
</ul> 
</font>

## Step 4: Let's create two Dashboards

We will create two dashboards that use the analysis that we have done in this tutorial.  

The first dashboard will present a list of the students and a synchronized chart of their gpa, number of semesters in the university and the cluster that they belong according to the sequence mining analysis.

Before runing the dashboard you need to install the following libraries in your R:

<pre>
install.packages(devtools)
devtools::install_github("rstudio/crosstalk")
devtools::install_github("datavisyn/lineup_htmlwidget")
devtools::install_github("jcheng5/d3scatter")
install.packages(TraMineR)
</pre>

The following is the code of student dashboard:

### First Dashboard code
<pre>
---
title: "Student Explorer"
output: 
  flexdashboard::flex_dashboard:
    orientation: columns
    vertical_layout: fill
  
---

```{r setup, include=FALSE}
library(flexdashboard)
library(crosstalk)
library(lineupjs)
library(d3scatter)
library(readxl)
library(dplyr)
library(reshape)
library(cluster)
library(TraMineR)

#Load the data
academic_data = read_excel("AnonymizedAcademicData.xlsx")
common_courses=c("c66fa9f7","a65b6acf","87824b85","1b40bae0","47f79e31","ee143666","0ad99f1e","44e3b71e","5647927e","2c39cee7","3f0c9401","45480b29","44ecaefc","5fcb1de4","dd6dd97d","3ef98742")

selected_data =
  academic_data %>%
  filter(course_code %in% common_courses) %>%
  filter(semester %in% c("1S","2S")) %>% 
  filter(year > 2000)

student_gpa = 
  selected_data %>%
  group_by(student_code) %>%
  summarise(
    gpa = mean(grade, na.rm = TRUE)
  ) 

student_semesters= selected_data %>% 
  mutate(semester_id=paste(year,semester,sep="_")) %>% 
  group_by(student_code,semester_id) %>%
  summarise(
    semesters=n()
  ) %>%
  group_by(student_code) %>%
  summarise(
    semesters=n()
  ) 

academic_sequence=selected_data %>% 
  mutate(sequence_id=paste(student_code,year,semester,sep="_")) %>%
  filter(semester %in% c("1S","2S")) %>% 
  filter(year > 2000)

semesters = academic_sequence %>%
  group_by(student_code,year,semester) %>%
  summarize(
    sequence_id=max(sequence_id),
    size= n(),
    courses = paste(as.character(course_code), collapse = ';')
  )

get_semester_order=function(sequence_id)
{
  components<-strsplit(sequence_id, "_")
  id<-components[[1]][1]
  year<-components[[1]][2]
  semester<-components[[1]][3]
  records<-academic_sequence[academic_sequence$student_code %in% c(id),]
  firstyear<-min(records$year)
  sem<-min(records[records$year==firstyear,]$semester)
  nofSemesters=strtoi(year)-strtoi(firstyear)
  nofSemesters=(nofSemesters*2)+1
  if(sem %in% c("1S") & semester %in% c("2S")){
    nofSemesters=nofSemesters+1
  }
  if(sem %in% c("2S") & semester %in% c("1S")){
    nofSemesters=nofSemesters-1
  }
  nofSemesters
}

semesters$semester_id=sapply(semesters$sequence_id,get_semester_order)

sequence_data=semesters %>% 
  group_by(student_code) %>%
  summarize(
    sequence = paste(as.character(size), collapse = '-')
  )

sqd=seqdef(sequence_data$sequence)

submat = seqsubm(sqd, method = "TRATE")
dist.om1 = seqdist(sqd, method = "OM", indel = 1, sm = submat)
clusterward1 = agnes(dist.om1, diss = TRUE, method = "ward")
cl1.4 = cutree(clusterward1, k = 4)
cl1.4fac = factor(cl1.4, labels = paste("Type", 1:4))

sequence_data$cluster=cl1.4fac
student_data = merge(student_gpa,student_semesters,by="student_code")
student_data = merge(student_data,sequence_data,by="student_code")
student_data = student_data %>% select(student_code,gpa,semesters,cluster)
sd = SharedData$new(student_data)

```

Column {data-width=650}
-----------------------------------------------------------------------

### List of Students

```{r}

lineup(sd)
```

Column {data-width=650}
-----------------------------------------------------------------------

### GPA vs Semester vs Cluster

```{r}
d3scatter(sd,~gpa, ~semesters,~cluster)
```

</pre>

The code for the second course dashboard is:

### Second dashboard code
<pre>
---
title: "Course Explorer"
resource_files:
- AnonymizedAcademicData.xlsx
runtime: shiny
output:
  flexdashboard::flex_dashboard:
    orientation: columns
    vertical_layout: fill
---

```{r setup, include=FALSE}
library(flexdashboard)
library(readxl)
library(dplyr)
library(reshape)
library(cluster)
library(reshape)
library(data.table)
library(ppclust)
library(plotly)

academic_data = read_excel("AnonymizedAcademicData.xlsx")
common_courses=c("c66fa9f7","a65b6acf","87824b85","1b40bae0","47f79e31","ee143666","0ad99f1e","44e3b71e","5647927e","2c39cee7","3f0c9401","45480b29","44ecaefc","5fcb1de4","dd6dd97d","3ef98742")

selected_data =
  academic_data %>%
  filter(course_code %in% common_courses) %>%
  filter(semester %in% c("1S","2S")) %>% 
  filter(year > 2000)

student_gpa = 
  selected_data %>%
  group_by(student_code) %>%
  summarise(
    gpa = mean(grade)
  )

get_gpa=function(code)
{
  gpa<- student_gpa %>% filter(student_code %in% c(code))
  gpa$gpa
}

selected_data$student_gpa=sapply(selected_data$student_code,get_gpa)
selected_data = selected_data %>% mutate(adjusted_grade=grade-student_gpa)

course_beta = 
  selected_data %>%
  group_by(course_code) %>%
  summarise(
    beta = mean(adjusted_grade)
  ) 

get_beta_distribution= function(code)
{
  betas<- selected_data %>% filter(course_code %in% c(code))
  betas$adjusted_grade
}

casted_data = cast(selected_data,student_code~course_code,value='grade',mean)

cleaned_data = na.omit(casted_data)


nam = names(cleaned_data[,-1])
course_data= transpose(cleaned_data[,-1])
rownames(course_data)<-nam
res.fcm = fcm(course_data, centers=4)
as.data.frame(res.fcm$u)
res.fcm$course_code<-nam
```

Column {.sidebar}
-----------------------------------------------------------------------

### Chart A

```{r}
selectInput("course_code", label = "Select a course:", choices = common_courses, selected = "c66fa9f7")
```

Column {data-width=650}
-----------------------------------------------------------------------

### Chart B

```{r}
renderPlot({
hist(get_beta_distribution(input$course_code),main=paste("Beta Distribution for course",input$course_code),xlab="Adjusted Grade",breaks=20)
abline(v=mean(get_beta_distribution(input$course_code)),col="red")
})
```

### Chart C

```{r}
renderPlotly({
  data_clusters=res.fcm$u[input$course_code,]
  scores = list(
  "Course" = data_clusters
  )
p <- plot_ly(
    type = 'scatterpolar',
    r = data_clusters,
    theta = c('Cluster 1','Cluster 2','Cluster 3', 'Cluster 4'),
    fill = 'toself'
  ) %>%
  layout(
    polar = list(
      radialaxis = list(
        visible = T,
        range = c(0,1)
      )
    ),
    showlegend = F
  )
})
```


</pre>